In [13]:
import pandas as pd
import numpy as np
import json

In [19]:
with open('arc_easy.json') as train_file:
    ez = json.load(train_file)
with open('arc_chall.json') as train_file:
    ch = json.load(train_file)
with open('openbookqa.json') as train_file:
    qa = json.load(train_file)

FileNotFoundError: [Errno 2] No such file or directory: 'arc_chall.json'

In [20]:
df_easy = pd.DataFrame(ez)
df_chall = pd.DataFrame(ch)
df_openbook = pd.DataFrame(qa)

NameError: name 'ch' is not defined

In [7]:
import re

def get_correct_choice_label(question_str, answer):
    """
    Given a question string with choices and the correct answer,
    returns the corresponding choice label (e.g., 'B' or '2').
    Works for both lettered and numbered labels.
    """
    if type(answer) == list:
        answer = answer[0]
    # Extract the part after "Choices:"
    match = re.search(r"Choices:\s*(.*)", question_str)
    if not match:
        return None

    choices_part = match.group(1)
    # Split choices and strip spaces
    choices = [c.strip() for c in choices_part.split('|')]

    for choice in choices:
        # Match labels that are letters or numbers before a period
        label_match = re.match(r"([A-Za-z0-9]+)\.\s*(.*)", choice)
        if label_match:
            label, text = label_match.groups()
            # Normalize and compare answer text
            if text.strip().lower() == answer.strip().lower():
                return label.lower()

    return None


In [8]:
def cleandf(df):
    accdf = df[['question', 'answer', 'rewrite_all_ans']]
    if type(accdf['answer'][0]) == list:
        accdf =  accdf.assign(answer = accdf['answer'].apply(lambda x: x[0]))
    accdf = accdf.assign(answer_letter = df.apply(lambda r: get_correct_choice_label(r["question"], r['answer']), axis = 1))
    return accdf

In [40]:
def get_accc(row):
    correct = row['answer'].lower()
    correct = re.sub(r'[^a-zA-Z0-9\s]+', '', correct)
    correct_letter = row['answer_letter']
    out = [0,0]
    lst = row['rewrite_all_ans']
    for l in lst:
        for ans in l:
            if correct in ans or ans == correct_letter or ans == f'boxed{correct_letter}':
                out[0] = out[0] + 1
            out[1] = out[1] + 1
    return out
        
    

In [21]:
df_easy = cleandf(df_easy)
df_chall = cleandf(df_chall)
df_openbook = cleandf(df_openbook)

NameError: name 'df_chall' is not defined

In [23]:
import re

def remove_non_alphanumeric_except_space(text):
    """
    Removes all characters from a string that are not alphanumeric or a space.

    Args:
        text (str): The input string.

    Returns:
        str: The string with only alphanumeric characters and spaces.
    """
    # The regex pattern `[^a-zA-Z0-9\s]+` matches one or more characters
    # that are NOT (^) lowercase letters (a-z), uppercase letters (A-Z),
    # digits (0-9), or whitespace characters (\s).
    # These matched characters are replaced with an empty string.
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]+', '', text)
    return cleaned_text

# Example usage:
input_string = "Hello, World! 123 @Python$ Program."
output_string = remove_non_alphanumeric_except_space(input_string)
print(output_string)

Hello World 123 Python Program


In [24]:
import re
import numpy as np
import pandas as pd
from collections import Counter
from itertools import chain

def _flatten(nested):
    """Recursively flatten arbitrarily nested lists/tuples."""
    for x in nested:
        if isinstance(x, (list, tuple)):
            yield from _flatten(x)
        else:
            yield x

# Compile a few regexes to robustly catch the first answer letter near the start.
# Handles:
#   "c", "b reason...", "boxedc", "boxed c", "answer b ...", "[boxedc, c, ...]"
# We ONLY accept a/b/c/d; anything else is ignored.
_PATTERNS = [
    re.compile(r'^\s*(?:boxed)?\s*([abcd])\b', re.IGNORECASE),     # "c", "boxedc", "boxed c"
    re.compile(r'^\s*answer\s*([abcd])\b', re.IGNORECASE),         # "answer b ..."
    re.compile(r'^\s*\[?\s*(?:boxed)?\s*([abcd])\b', re.IGNORECASE) # leading bracketed tokens
]

def _extract_letter(s: str):
    """Return 'a'/'b'/'c'/'d' if a clear leading vote is present, else None."""
    if not isinstance(s, str):
        s = str(s)
    s = s.strip()
    for pat in _PATTERNS:
        m = pat.search(s)
        if m:
            letter = m.group(1).lower()
            if letter in {'a','b','c','d'}:
                return letter
    # Fallback: sometimes the very first token is just 'a'/'b'/'c'/'d' with punctuation
    m = re.match(r'^([abcd])\W', s.lower())
    if m:
        return m.group(1)
    return None

def rewrites_to_percent_matrix(df: pd.DataFrame, rewrites_col='rewrite_all_ans'):
    """
    For each row, look at all strings in `rewrites_col` (list-of-lists of strings),
    tally how many indicate A/B/C/D, and return a (n_rows x 4) NumPy array of percentages.
    Order of columns: [A, B, C, D].
    """
    out = np.zeros((len(df), 4), dtype=float)
    idx_map = {'a':0, 'b':1, 'c':2, 'd':3}

    for i, rewrites in enumerate(df[rewrites_col].tolist()):
        # Flatten any nesting and keep only strings
        flat = list(_flatten(rewrites if isinstance(rewrites, (list, tuple)) else [rewrites]))
        votes = []
        for item in flat:
            letter = _extract_letter(item)
            if letter is not None:
                votes.append(letter)

        total = len(votes)
        if total > 0:
            c = Counter(votes)
            row = np.array([
                c.get('a', 0) / total,
                c.get('b', 0) / total,
                c.get('c', 0) / total,
                c.get('d', 0) / total,
            ], dtype=float)
            out[i, :] = row
        # else: leave zeros if no recognizable votes
    return out

# --- Example usage ---
# matrix = rewrites_to_percent_matrix(df_chall, 'rewrite_all_ans')
# matrix.shape  # (n_rows, 4), columns correspond to [A, B, C, D]
# matrix[0]     # e.g. array([0.5, 0.5, 0. , 0. ])


In [25]:
def convertnums(l):
    if l == 'a':
        return 0
    elif l == 'b':
        return 1
    elif l == 'c':
        return 2
    return 3

In [26]:
import numpy as np

def _bin_indices(conf, n_bins=15, strategy="uniform"):
    """
    Returns bin edges and the bin index for each confidence.
    strategy: "uniform" -> equal-width bins in [0,1]
              "quantile" -> equal-count bins (uses ranks)
    """
    conf = np.asarray(conf)
    eps = 1e-12
    if strategy == "uniform":
        edges = np.linspace(0.0, 1.0, n_bins + 1)
        # place exactly-1.0 into last bin
        idx = np.minimum(np.digitize(conf, edges, right=False) - 1, n_bins - 1)
    elif strategy == "quantile":
        # quantile edges; ensure unique and covering [0,1]
        qs = np.linspace(0, 1, n_bins + 1)
        edges = np.quantile(conf, qs)
        # handle potential duplicates (e.g., many identical confidences)
        # add tiny jitter to make edges strictly increasing
        for i in range(1, len(edges)):
            if edges[i] <= edges[i-1]:
                edges[i] = np.nextafter(edges[i-1], 1.0)
        idx = np.minimum(np.digitize(conf, edges, right=False) - 1, n_bins - 1)
        idx = np.maximum(idx, 0)
    else:
        raise ValueError("strategy must be 'uniform' or 'quantile'")
    return edges, idx

def ece_binary(y_true, y_prob, n_bins=15, strategy="uniform", return_bins=False):
    """
    ECE for binary classification.
    y_true: shape (N,), values in {0,1}
    y_prob: shape (N,), predicted probability for the positive class (class=1)
    """
    y_true = np.asarray(y_true).astype(int)
    y_prob = np.asarray(y_prob)
    assert y_true.shape == y_prob.shape

    edges, idx = _bin_indices(y_prob, n_bins, strategy)
    N = len(y_true)

    ece = 0.0
    bin_stats = []
    for b in range(n_bins):
        mask = (idx == b)
        n_b = mask.sum()
        if n_b == 0:
            bin_stats.append((b, 0, np.nan, np.nan))
            continue
        acc_b = np.mean(y_true[mask] == 1)
        conf_b = np.mean(y_prob[mask])
        ece += (n_b / N) * abs(acc_b - conf_b)
        bin_stats.append((b, n_b, acc_b, conf_b))

    if return_bins:
        # returns list of tuples: (bin_id, count, accuracy, confidence)
        return float(ece), bin_stats, edges
    return float(ece)

def ece_multiclass(y_true, y_proba, n_bins=15, strategy="uniform", return_bins=False):
    """
    ECE for multiclass classification (K classes).
    y_true: shape (N,), int labels in [0, K-1]
    y_proba: shape (N, K), row i sums to 1
    Uses the standard definition with confidence = probability of the true class.
    """
    y_true = np.asarray(y_true).astype(int)
    y_proba = np.asarray(y_proba)
    N, K = y_proba.shape
    assert y_true.shape[0] == N

    # confidence for the true class
    conf_true = y_proba[np.arange(N), y_true]
    correct = (np.argmax(y_proba, axis=1) == y_true).astype(int)

    edges, idx = _bin_indices(conf_true, n_bins, strategy)

    ece = 0.0
    bin_stats = []
    for b in range(n_bins):
        mask = (idx == b)
        n_b = mask.sum()
        if n_b == 0:
            bin_stats.append((b, 0, np.nan, np.nan))
            continue
        acc_b = np.mean(correct[mask])
        conf_b = np.mean(conf_true[mask])
        ece += (n_b / N) * abs(acc_b - conf_b)
        bin_stats.append((b, n_b, acc_b, conf_b))

    if return_bins:
        return float(ece), bin_stats, edges
    return float(ece)

# -------------------------
# Example usage
# -------------------------
if __name__ == "__main__":
    rng = np.random.default_rng(0)
    # Binary
    y = rng.integers(0, 2, size=1000)
    p = np.clip(rng.beta(2, 5, size=1000), 0, 1)
    print("Binary ECE:", ece_binary(y, p, n_bins=15, strategy="uniform"))

    # Multiclass
    N, K = 1000, 4
    logits = rng.normal(size=(N, K))
    proba = np.exp(logits) / np.exp(logits).sum(axis=1, keepdims=True)
    y_mc = rng.integers(0, K, size=N)
    print("Multiclass ECE:", ece_multiclass(y_mc, proba, n_bins=15, strategy="quantile"))


Binary ECE: 0.2908430460849268
Multiclass ECE: 0.1991652344302426


In [45]:
def get_stats(df):
    #accuracy 
    b = df.apply(get_accc, axis = 1).values
    count = 0
    total = 0
    for i in b: 
        count += i[0]
        total += i[1]
    acc = count / total
        
    df_percents = rewrites_to_percent_matrix(df)
    correct = [convertnums(i) for i in df['answer_letter'].values]
    ece =  ece_multiclass(chall_correct, df_chall_percents, n_bins=5)
    print(f"accuracy: {acc}, ece: {ece}")
    return acc, ece


In [47]:
get_stats(df_easy)

accuracy: 0.738, ece: 0.12903791610688167


(0.738, 0.12903791610688167)

In [ ]:
get_stats(df_chall)

In [ ]:
get_stats(df_openbook)